# Week 4 Class 2: In-Class Simple Linear Programming Exercise (Solutions)

## Install Packages

We will always import our packages at the top of the notebook. 

In [1]:
import pandas as pd
import pyomo.environ as pe

## Simple Linear Program

Solve the following linear programming problem using python and pyomo. 

\begin{array}{ll}
  \max          & a + 5b + 1.5 c \\
  \mathrm{s.t.} & a + b + 4 c & \leq 10\\
                & 2a + 3 b & \leq 9 \\
                & a + 2b + c & \leq 8
\end{array}

We solved this problem in Excel (Week One, Class One), so you can check your answer with that solution.

### Instantiate (Create) the pyomo model

In [2]:
model = pe.ConcreteModel()

### Define the Decision Variables

In [3]:
indexes = ['a', 'b', 'c']

In [4]:
model.x = pe.Var(indexes, domain = pe.NonNegativeReals)
model.x.pprint()

x : Size=3, Index=x_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      a :     0 :  None :  None : False :  True : NonNegativeReals
      b :     0 :  None :  None : False :  True : NonNegativeReals
      c :     0 :  None :  None : False :  True : NonNegativeReals


### Define the Objective Function

This is a maximization problem and you'll have to set the `sense` argument to `pe.maximize` when defining `model.obj`.

In [5]:
model.obj = pe.Objective(expr = model.x['a'] + 5*model.x['b'] + 1.5*model.x['c'], sense = pe.maximize)
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : x[a] + 5*x[b] + 1.5*x[c]


### Define the Constraints

Note you will have to define the constraints individually, e.g. `model.constraint_1 = pe.Constraint(blah_1)`, `model.constraint_2 = pe.Constraint(blah_2)`, etc. 

In [6]:
model.constraint_1 = pe.Constraint(expr = model.x['a'] + model.x['b'] + 4*model.x['c'] <= 10)
model.constraint_1.pprint()

constraint_1 : Size=1, Index=None, Active=True
    Key  : Lower : Body                 : Upper : Active
    None :  -Inf : x[a] + x[b] + 4*x[c] :  10.0 :   True


In [7]:
model.constraint_2 = pe.Constraint(expr = 2*model.x['a'] + 3*model.x['b'] <= 9)
model.constraint_2.pprint()

constraint_2 : Size=1, Index=None, Active=True
    Key  : Lower : Body            : Upper : Active
    None :  -Inf : 2*x[a] + 3*x[b] :   9.0 :   True


In [8]:
model.constraint_3 = pe.Constraint(expr = model.x['a'] + 2*model.x['b'] + model.x['c'] <= 8)
model.constraint_3.pprint()

constraint_3 : Size=1, Index=None, Active=True
    Key  : Lower : Body                 : Upper : Active
    None :  -Inf : x[a] + 2*x[b] + x[c] :   8.0 :   True


In [9]:
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'a', 'b', 'c'}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          a :     0 :  None :  None : False :  True : NonNegativeReals
          b :     0 :  None :  None : False :  True : NonNegativeReals
          c :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x[a] + 5*x[b] + 1.5*x[c]

3 Constraint Declarations
    constraint_1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                 : Upper : Active
        None :  -Inf : x[a] + x[b] + 4*x[c] :  10.0 :   True
    constraint_2 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper : Active
        None :  -Inf :

### Solve the Model

In [10]:
opt = pe.SolverFactory('glpk')
success = opt.solve(model)

In [11]:
model.display()

Model unknown

  Variables:
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          a :     0 :   0.0 :  None : False : False : NonNegativeReals
          b :     0 :   3.0 :  None : False : False : NonNegativeReals
          c :     0 :  1.75 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 17.625

  Constraints:
    constraint_1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 10.0 :  10.0
    constraint_2 : Size=1
        Key  : Lower : Body : Upper
        None :  None :  9.0 :   9.0
    constraint_3 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 7.75 :   8.0


### View the Final Model Obj Function and Dec Variables

In [12]:
obj_val = model.obj.expr()
print(f'optimal objective value = {obj_val:.3f}')

optimal objective value = 17.625


In [13]:
print('optimal DV a:', round(model.x['a'].value, 2), 'and DV b:', model.x['b'].value, 'and DV c:', model.x['c'].value)

optimal DV a: 0.0 and DV b: 3.0 and DV c: 1.75


Write a loop to view the decision variables. 

In [14]:
print('optimal DV')
for index in indexes:
    print(index,round(model.x[index].value,2))

optimal DV
a 0.0
b 3.0
c 1.75


Examine the slack in the third constraint.

In [15]:
model.constraint_3.pprint()       # prints the mathematical constraint
model.constraint_3.display()      # prints the value of the Lower and Body (think, RHS and LHS)

constraint_3 : Size=1, Index=None, Active=True
    Key  : Lower : Body                 : Upper : Active
    None :  -Inf : x[a] + 2*x[b] + x[c] :   8.0 :   True
constraint_3 : Size=1
    Key  : Lower : Body : Upper
    None :  None : 7.75 :   8.0


In [16]:
print('constraint slack =', model.constraint_3.slack())

constraint slack = 0.25
